In [1]:
from functions.backtesting_functions import *

# Read Files

In [2]:
INPUT_DIRECTORY = './input/sp500_log_returns.rds'

SnP500_lr_r_obj = read_rds(INPUT_DIRECTORY)
SnP500_lr_index_r_obj = SnP500_lr_r_obj['attributes']['index']['data']
SnP500_lr_timestamp_index_r_obj = [datetime.utcfromtimestamp(i).strftime('%Y-%m-%d') for i in SnP500_lr_index_r_obj]
SnP500_lr__data_r_obj = SnP500_lr_r_obj['data']
df_SnP500_lr = pd.DataFrame(data={'Forecast': SnP500_lr__data_r_obj}, index=SnP500_lr_timestamp_index_r_obj)


In [3]:
OUTPUT_DIRECTORY = './output'
FORECASTS_BC_SA = {}

for file in os.listdir(OUTPUT_DIRECTORY):
    if not file.startswith('.'):
                    
        all_forecasts = []
        for file_in_dir in os.listdir(os.path.join(OUTPUT_DIRECTORY, file)):
            if not file_in_dir.startswith('.'):

                path = os.path.join(OUTPUT_DIRECTORY, file, file_in_dir)
                r_obj = read_rds(path)

                index_r_obj = r_obj['attributes']['index']['data']
                timestamp_index_r_obj = [datetime.utcfromtimestamp(i).strftime('%Y-%m-%d') for i in index_r_obj]
                data_r_obj = r_obj['data']

                df_forecast = pd.DataFrame(data={'Forecast': data_r_obj}, index=timestamp_index_r_obj)
                all_forecasts.append(df_forecast)
        
        df_all_forecasts = pd.concat(all_forecasts).sort_index()
        FORECASTS_BC_SA[file] = df_all_forecasts

In [4]:
FORECASTS_BC_SA.keys()

dict_keys(['arima_base_case', 'arima-egarch_ged_1000', 'arima-garch_base_case'])

### Case 1: ARIMA(p,1,q) 1000 and ARIMA(p,1,q)-SGARCH(1,1) 1000 (Base Case)

In [5]:
def cases_df(FORECASTS_BC_SA, df_SnP500_lr, case_1):
    df_case_strat = pd.DataFrame(columns=case_1, index=FORECASTS_BC_SA['arima-garch_base_case'].index)

    for strat in case_1:

        res = pd.DataFrame()

        res['forecast'] = FORECASTS_BC_SA[strat]
        res['SnP500_lr'] = df_SnP500_lr.loc[FORECASTS_BC_SA['arima-garch_base_case'].index]
        res['SnP500_lr'] = np.exp(res['SnP500_lr'])-1 # converting to simple returns

        res = get_position(res)

        # calculating equity line

        strategy = (res['SnP500_lr'] * res['position'].shift(1))
        strategy = (1+strategy.fillna(0)).cumprod()

        df_case_strat[strat] = strategy
        df_case_strat[f'{strat}_forecast'] = res['forecast']

    df_case_strat['buy_n_hold'] = (1 + res['SnP500_lr'].fillna(0)).cumprod()
    df_case_strat['buy_n_hold_forecast'] = res['SnP500_lr']

    case_1.append('buy_n_hold')

    return df_case_strat, case_1

In [6]:
case_1 = ['arima_base_case', 'arima-garch_base_case']
df_case_strat_case_1, case_1 = cases_df(FORECASTS_BC_SA, df_SnP500_lr, case_1)

In [7]:
performance_metrics_case_1(case_1[0], case_1[1], df_case_strat_case_1, case_1)

/Users/kamilkashif/Documents/Python/rr-project/functions/backtesting_functions.py:51: RuntimeWarning: divide by zero encountered in divide
  return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
/Users/kamilkashif/Documents/Python/rr-project/functions/backtesting_functions.py:51: RuntimeWarning: divide by zero encountered in divide
  return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


,,MAE,MSE,RMSE,MAPE,ARC(%),ASD(%),MD,IR*(%),IR**(%)
,S&P 500,None,None,None,None,6.48,18.77,55.76,34.52,4.01
,arima_base_case,7.72e-03,1.47e-04,1.21e-02,inf,-4.51,13.97,58.26,-32.28,-2.50
,arima-garch_base_case,2.11e+138,1.54e+280,1.24e+140,inf,-0.36,10.66,42.37,-3.42,-0.03


### Case 2: ARIMA(p,1,q) 1000 and ARIMA(p,1,q)-SGARCH(1,1) 1000 and ARIMA(p,1,q)-EGARCH(1,1) 1000

In [8]:
case_2 = ['arima_base_case', 'arima-garch_base_case', 'arima-egarch_ged_1000']
df_case_strat_case_2, case_2 = cases_df(FORECASTS_BC_SA, df_SnP500_lr, case_2)

In [9]:
performance_metrics_case_2(case_2[0], case_2[1], case_2[2], df_case_strat_case_2, case_2)

/Users/kamilkashif/Documents/Python/rr-project/functions/backtesting_functions.py:51: RuntimeWarning: divide by zero encountered in divide
  return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
/Users/kamilkashif/Documents/Python/rr-project/functions/backtesting_functions.py:51: RuntimeWarning: divide by zero encountered in divide
  return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
/Users/kamilkashif/Documents/Python/rr-project/functions/backtesting_functions.py:51: RuntimeWarning: divide by zero encountered in divide
  return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


,,MAE,MSE,RMSE,MAPE,ARC(%),ASD(%),MD,IR*(%),IR**(%)
,S&P 500,None,None,None,None,6.48,18.77,55.76,34.52,4.01
,arima_base_case,7.72e-03,1.47e-04,1.21e-02,inf,-4.51,13.97,58.26,-32.28,-2.50
,arima-garch_base_case,2.11e+138,1.54e+280,1.24e+140,inf,-0.36,10.66,42.37,-3.42,-0.03
,arima-egarch_ged_1000,7.57e-03,1.42e-04,1.19e-02,inf,1.32,12.62,33.12,10.42,0.41


### Case 3: ARIMA(p,1,q) 500 and ARIMA(p,1,q)-SGARCH(1,1) 500 / ARIMA(p,1,q) 1000 and ARIMA(p,1,q)-SGARCH(1,1) 1000 / ARIMA(p,1,q) 1500 and ARIMA(p,1,q)-SGARCH(1,1) 1500

In [10]:
case_3 = ['arima_base_case', 'arima-garch_base_case', 'arima-egarch_ged_1000']

### Case 4: ARIMA 1000 and SGARCH.GED 1000 and SGARCH.SNORM 1000 and SGARCH.SSTD 1000 and SGARCH.SGED 1000

In [11]:
case_4 = ['arima_base_case', 'arima-garch_base_case', 'arima-egarch_ged_1000']

In [12]:
# In general, the rule for going long (buy) or short (sell) is as follows: 
# if the forecasted log return is positive at time t + 1, we go long (buy stocks) at time t (direction would be +1); 
# if the forecasted log return is negative at time t + 1, we go short (sell stocks) at time t (direction would be −1); 
# and if the forecasted direction at time t+1 is the same as at time t, then there are no changes.

# The initial investment is assumed to be $1259.92 at the beginning.

In [13]:
# Add cases
# Check why strange results
# Add R ggplot graph code